In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
import tensorflow as tf
tf.random.set_seed(777) #하이퍼파라미터 튜닝을 위해 실행시 마다 변수가 같은 초기값 가지게 하기
import numpy as np

In [ ]:
from google.colab import files
data = files.upload()

In [ ]:
train_df = pd.read_excel('cutcut1.xlsx', sheet_name='train')
test_df = pd.read_excel('cutcut1.xlsx', sheet_name='test')
x_train = train_df.drop(['당첨최소가점'], axis=1)
x_test = test_df.drop(['당첨최소가점'], axis=1)
y_train = train_df[['당첨최소가점']]
y_test = test_df[['당첨최소가점']]
print(x_train.head())

In [ ]:
print(x_train.columns)


In [ ]:
transformer = make_column_transformer(
    (OneHotEncoder(), []),
    remainder='passthrough')
transformer = make_pipeline(transformer, MinMaxScaler())
transformer.fit(x_train)

x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)


In [ ]:
print(x_train.shape)
print(y_train.shape)


In [ ]:
input = tf.keras.layers.Input(shape=(9,))
net = tf.keras.layers.Dense(units=426, activation='relu')(input)
net = tf.keras.layers.Dense(units=426, activation='relu')(net)
net = tf.keras.layers.Dense(units=1)(net)
model = tf.keras.models.Model(input, net)


In [ ]:
##########모델 학습 및 검증

loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()
train_summary_writer = tf.summary.create_file_writer('tensorboard/gradecut_model/train') #텐서 보드
test_summary_writer = tf.summary.create_file_writer('tensorboard/gradecut_model/test')


epochs = 5000
for epoch_index in range(epochs):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss_value = loss(y_train, predictions)
    gradients = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss_value)

    predictions = model(x_test)
    loss_value = loss(y_test, predictions)
    test_loss.update_state(loss_value)

    print('epoch: {}/{}, train loss: {:.4f}, test loss: {:.4f}'.format(
        epoch_index + 1, epochs, train_loss.result().numpy(), test_loss.result().numpy()))

    with train_summary_writer.as_default(): #텐서 보드
        for v in model.trainable_variables:
            tf.summary.histogram(v.name, v, step=epoch_index)
        tf.summary.scalar('loss', train_loss.result(), step=epoch_index)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch_index)

    train_loss.reset_states()
    test_loss.reset_states()

'''
#내장 루프
model.compile(loss='mean_squared_error', optimizer='adam')
#model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), callbacks=[tf.keras.callbacks.TensorBoard(log_dir='tensorboard/hyundae_car_price_regression_model', histogram_freq=1)])
'''

##########모델 예측

x_test = [
    [59.9883,0.151,0.095,0.285,25,448,62,37,8.19]
]
x_test = pd.DataFrame(x_test, columns=['평형', '단지에서 역까지의거리(km)(최대,최소 구해서 평균)','공원까지의 거리(km)','학교까지의 거리(km)', '강남까지의 소요시간(분)','도급순위','단지세대수', '공급세대수', '경쟁률' ])
x_test = transformer.transform(x_test)


y_predict = model.predict(x_test)

print(y_predict[0][0]) #